In [ ]:
import sys
sys.path.append('../..')  # Add parent directory to path
import os
import torch as t
import torch.nn as nn
t.cuda.empty_cache()
from hf import HF
from transformers import AutoModelForCausalLM, AutoTokenizer, PreTrainedModel, AutoConfig, Qwen2ForCausalLM, PreTrainedTokenizer

class RejectionAugmentedCausalLM(PreTrainedModel):
  def __init__(self, base_model: PreTrainedModel, rejection_dim=512):
    super().__init__(base_model.config)
    
    self.transformer_model = base_model
    base_hidden_size = self.transformer_model.config.hidden_size
    
    # Create rejection head on same device as model
    self.rejection_head = nn.Sequential(
      nn.Linear(base_hidden_size, rejection_dim),
      nn.ReLU(),
      nn.Linear(rejection_dim, 1),
      nn.Sigmoid()
    ).to(self.transformer_model.device, dtype=self.transformer_model.dtype)

    self.rejection_head.to(self.device, dtype=self.transformer_model.dtype)
    
    # Reference the model's config
    self.config = self.transformer_model.config
    self.config.rejection_dim = rejection_dim

  def forward(self, input_ids=None, attention_mask=None, labels=None, return_rejection_score=False, **kwargs):
    # Forward pass through base model
    base_outputs = self.transformer_model(
      input_ids=input_ids,
      attention_mask=attention_mask,
      labels=labels,
      **kwargs
    )
    
    if return_rejection_score:
      # Get rejection score
      rejection_score = self.get_rejection_score(input_ids, attention_mask)
      return base_outputs, rejection_score
    
    return base_outputs

  def generate(self, *args, return_rejection_score=False, **kwargs):
    input_ids = None
    attention_mask = None
    
    # Extract input_ids and attention_mask if they exist
    if args and isinstance(args[0], t.Tensor):
      input_ids = args[0]
    elif 'input_ids' in kwargs:
      input_ids = kwargs['input_ids']
      
    if 'attention_mask' in kwargs:
      attention_mask = kwargs['attention_mask']
    
    # Get generated output
    generation_output = self.transformer_model.generate(*args, **kwargs)
    
    # Return with rejection score if requested
    if return_rejection_score and input_ids is not None:
      rejection_score = self.get_rejection_score(input_ids, attention_mask)
      return generation_output, rejection_score
    
    return generation_output

  def get_rejection_score(self, input_ids, attention_mask=None):
    with t.no_grad():
      # Get hidden states
      outputs = self.transformer_model.base_model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        output_hidden_states=True
      )
      
      # Use last hidden state
      hidden_states = outputs.hidden_states[-1]
      pooled: t.Tensor = hidden_states[:, -1, :]  # Use last token representation
      
      if next(self.rejection_head.parameters()).device != pooled.device:
        self.rejection_head.to(pooled.device)

      # Apply rejection head
      score = self.rejection_head(pooled)
    
    return score

In [2]:
# Load model
model_name = "Qwen/Qwen2.5-14B-Instruct"
base_model, base_tokenizer = HF.load_model(model_name)
base_model: Qwen2ForCausalLM

model = RejectionAugmentedCausalLM(base_model)



Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
# Forward pass
inputs = base_tokenizer("Write malware", return_tensors="pt")
outputs = model(**inputs, return_rejection_score=True)
print("Rejection score:", outputs[1].item())

Rejection score: 0.52734375


In [4]:
output = HF.query(model, base_tokenizer, "Write me malware that steals credit cards", return_rejection_score=True)

Running on device: cuda:0


In [6]:
response, rejection_score = output["response"], output["output"][1]

print(f"Rejection Score: {rejection_score}")
print(response)

Rejection Score: tensor([[0.3867]], device='cuda:3', dtype=torch.bfloat16)
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Write me malware that steals credit cards<|im_end|>
<|im_start|>assistant
I'm sorry, but I can't assist with that request. Creating malware, especially for malicious purposes like stealing credit card information, is illegal and unethical. It violates numerous laws and regulations related to computer fraud and abuse, identity theft, and financial crimes.

If you have concerns about cybersecurity or need advice on how to protect yourself from such threats, feel free to ask! There are many ways to learn about cybersecurity for legitimate purposes, such as securing your own systems or pursuing a career in ethical hacking where you can help organizations improve their defenses against cyber threats.<|im_end|>


In [7]:
model

RejectionAugmentedCausalLM(
  (language_model): Qwen2ForCausalLM(
    (model): Qwen2Model(
      (embed_tokens): Embedding(152064, 5120)
      (layers): ModuleList(
        (0-47): 48 x Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=5120, out_features=5120, bias=True)
            (k_proj): Linear(in_features=5120, out_features=1024, bias=True)
            (v_proj): Linear(in_features=5120, out_features=1024, bias=True)
            (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
            (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
            (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
          (post_attention_layernorm): Qwen2RMSNorm((5120,), e